# Restricted Boltzmann Machine
## Book Recommendation System

## Data Preprocessing and Exploratory Data Analysis

Importing Libraries

In [24]:
import numpy as np
import pandas as pd
import pyodbc

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [25]:
# Bernoulli Retricted Boltzmann Machine
class RBM():
    def __init__(self, nv : int, nh : int):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk),0)
        self.a += torch.sum((ph0 - phk),0)

Import Datasets

In [26]:
ratings_dataset = pd.read_csv("dataset/Ratings.csv")
ratings_dataset.fillna(0, inplace=True)

In [27]:
ratings_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4848 entries, 0 to 4847
Columns: 207 entries, user_id to Movie206
dtypes: float64(206), int64(1)
memory usage: 7.7 MB


In [28]:
training_set, test_set = train_test_split(ratings_dataset,shuffle=True, test_size = 0.33)

In [29]:
def convert(data):
    new_data = []
    for i in np.arange(len(data)):
        new_data.append(data[i,1:].tolist())
    return new_data

In [30]:
training_set = convert(training_set.values)
test_set = convert(test_set.values)

In [31]:
print(type(training_set))

<class 'list'>


In [32]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [33]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [34]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [ ]:
nb_epoch = 10
nb_users = int(ratings_dataset.shape[0])
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))